In [1]:
from tkinter import *
from tkinter import colorchooser
from PIL import ImageTk, Image
%run ./base_exercises.ipynb
%run ./exercise_data.ipynb

# Global variable to keep track of the current exercise index
# When moving to the next/prev exercise, this index changes accordingly
current_index = 0

# Dictionary to store text colors for each exercise bc each might have different colors 
text_colors = {}

# Dictionary to store current colors for different parts of the exercise details
current_colors = {
    'title_label': 'black',
    'name_label': 'black',
    'name': 'black',
    'description_label': 'black',
    'description': 'black',
    'duration_label': 'black',
    'duration': 'black',
    'difficulty_label': 'black',
    'difficulty': 'black',
    'muscles_label': 'black',
    'muscles': 'black',
    'instructions_label': 'black',
    'instructions': 'black',
    'textcolor': 'red'
}

# Create a Tkinter window
window = Tk()
window.title("Exercise Details")
icon = PhotoImage(file='../Shared Components/Assets/workout_icon.png')
window.iconphoto(True, icon)
# window.config(bg="#05473F")


# Function to display exercise details in the window
def display_exercise_details():
    # Access global var to get current index
    global current_index
    
    # Get the current exercise to retrieve the details to be displayed
    exercise = exercises[current_index]
    
    # Initialize color dictionary for current exercise if not already done
    if exercise.name not in text_colors:
        text_colors[exercise.name] = current_colors.copy() # Creates a new dict w/ default colors for the current exercise
    
    # Clears previous labels & texts
    for widget in window.winfo_children():
        widget.destroy()
        
    # Function to choose color
    def choose_color(label_key, label):
        color_code = colorchooser.askcolor(title="Choose Text Color")[1]
        if color_code:
            label.config(fg=color_code)
            text_colors[exercise.name][label_key] = color_code
            # Update current colors to reflect the new choice for subsequent exercises
            current_colors[label_key] = color_code

    # Create labels to display exercise details
    def create_label(text, row, column, font=("Verdana", 15, 'bold'), label_key=None, colspan=1, width=None, height=None):
        # Determining the text color to use for this label
        fg_color = text_colors[exercise.name].get(label_key, current_colors[label_key])
        
        # Creates the label w/ the specified text, font, & color
        label = Label(window, 
                      text=text, 
                      font=font, 
                      bg='#AAC7D7', 
                      fg=fg_color, 
                      cursor="hand2", 
                      borderwidth=10, 
                      relief=RAISED, 
                      padx=10, 
                      pady=10,
                      width=width,
                      height=height
                      )
        
        # Places the label in the grid layout at the specified row/col
        label.grid(row=row, column=column, columnspan=colspan, sticky="w", padx=5, pady=5)
        
        # If label_key provided, makes the label clickable to change its color
        if label_key:
            label.bind("<Button-1>", lambda e: choose_color(label_key, label))
            
        # Returns the created label
        return label

    # Title Label
    create_label("Fitness for All!", 0, 0, label_key='title_label', colspan=4, width=100)   
        
    # Name Label
    create_label("Name:", 1, 0, label_key='name_label')
    create_label(exercise.name, 1, 1, font=("Verdana", 15, 'bold'), label_key='name')

    # Description Label
    create_label("Description:", 2, 0, label_key='description_label')
    create_label(exercise.description, 2, 1, label_key='description')
    
    # Duration Label
    create_label("Duration:", 3, 0, label_key='duration_label')
    create_label(str(exercise.duration) + " seconds", 3, 1, label_key='duration')
    
    # Difficulty Label
    create_label("Difficulty:", 4, 0, label_key='difficulty_label')
    create_label(exercise.difficulty, 4, 1, label_key='difficulty')
    
    # Muscle Groups Label
    create_label("Muscles Worked:", 5, 0, label_key='muscles_label')
    muscle_groups_str = ", ".join(exercise.muscle_groups)
    create_label(muscle_groups_str, 5, 1, label_key='muscles')
    
    # Instructions Label
    instructions_label = create_label("Instructions:", 6, 0, label_key='instructions_label')
    instructions_text = Text(window, height=6, width=40, wrap=WORD, 
                             fg=text_colors[exercise.name]['instructions'], 
                             cursor="hand2", 
                             borderwidth=10, 
                             relief=SUNKEN
                             )
    instructions_text.insert(END, exercise.instructions)
    instructions_text.config(state=DISABLED, font=("Verdana", 15, 'bold'))  # Make the text entry field read-only
    instructions_text.grid(row=6, column=1, sticky="w")
    instructions_text.bind("<Button-1>", lambda e: choose_color('instructions', instructions_text))
    
    # Click on text Label
    create_label("Click on any text to change its color :D", 7, 1, font=("Verdana", 13), label_key='textcolor', colspan=1)
    
    

    # Load/Displaying the Workout Images
    img = Image.open(exercise.image_path)
    img = img.resize((300, 300), Image.LANCZOS)
    photo_img = ImageTk.PhotoImage(img)  # Converts PIL image object into PhotoImage 
    img_label = Label(window, image=photo_img)
    img_label.image = photo_img  # Keeps a reference to avoid garbage collection
    img_label.grid(row=1, column=2, rowspan=6)
    
    # Creating Navigation Buttons
    prev_btn = Button(window, text="Previous", 
                      command=show_prev_exercise,
                      width=15,
                      height=5,
                      cursor="left_ptr",
                      relief=RAISED,
                      borderwidth=10,
                      font=("Verdana", 13)
                      )
    prev_btn.grid(row=7, column=0, sticky="w", padx=5, pady=30)
    
    next_btn = Button(window, text="Next", 
                      command=show_next_exercise,
                      width=15,
                      height=5,
                      cursor="right_ptr",
                      relief=RAISED,
                      borderwidth=10,
                      font=("Verdana", 13),
                      activebackground="yellow"
                      )
    next_btn.grid(row=7, column=1, sticky="e")
    
# Function to show previous exercise
def show_prev_exercise():
    global current_index
    current_index = (current_index - 1) % len(exercises)
    display_exercise_details()

# Function to show next exercise
def show_next_exercise():
    global current_index
    current_index = (current_index + 1) % len(exercises)
    display_exercise_details()    

# Displaying the first exercise
display_exercise_details()

# Configure grid row and column weights to make them expandable
for row in range(8):
    window.grid_rowconfigure(row, weight=1)
for column in range(3):
    window.grid_columnconfigure(column, weight=1)

window.mainloop()


In [1]:
from tkinter import *
from tkinter import colorchooser
from PIL import ImageTk, Image
%run ./base_exercises.ipynb
%run ./exercise_data.ipynb

# Global variable to keep track of the current exercise index
# When moving to the next/prev exercise, this index changes accordingly
current_index = 0

# Dictionary to store text colors for each exercise bc each might have different colors 
text_colors = {}

# Dictionary to store current colors for different parts of the exercise details
current_colors = {
    'title_label': 'black',
    'name_label': 'black',
    'name': 'black',
    'description_label': 'black',
    'description': 'black',
    'duration_label': 'black',
    'duration': 'black',
    'difficulty_label': 'black',
    'difficulty': 'black',
    'muscles_label': 'black',
    'muscles': 'black',
    'instructions_label': 'black',
    'instructions': 'black',
    'textcolor': 'red'
}
window = Tk()
window.title("Background Test")
window.geometry("800x700")

image_path = PhotoImage(file="../Shared Components/Assets/puzzle_bg.png")
bg_image = Label(window, image=image_path)
bg_image.place(relheight=1, relwidth=1)

text_label = Label(window, text="Welcome to my app!", font=("Georgia", 24))
text_label.pack()

# Function to display exercise details in the window
def display_exercise_details():
    # Access global var to get current index
    global current_index
    
    # Get the current exercise to retrieve the details to be displayed
    exercise = exercises[current_index]
    
    # Initialize color dictionary for current exercise if not already done
    if exercise.name not in text_colors:
        text_colors[exercise.name] = current_colors.copy() # Creates a new dict w/ default colors for the current exercise
    
    # Clears previous labels & texts
    for widget in window.winfo_children():
        widget.destroy()
        
    # Function to choose color
    def choose_color(label_key, label):
        color_code = colorchooser.askcolor(title="Choose Text Color")[1]
        if color_code:
            label.config(fg=color_code)
            text_colors[exercise.name][label_key] = color_code
            # Update current colors to reflect the new choice for subsequent exercises
            current_colors[label_key] = color_code

    # Create labels to display exercise details
    def create_label(text, row, column, font=("Verdana", 15, 'bold'), label_key=None, colspan=1, width=None, height=None):
        # Determining the text color to use for this label
        fg_color = text_colors[exercise.name].get(label_key, current_colors[label_key])
        
        # Creates the label w/ the specified text, font, & color
        label = Label(window, 
                      text=text, 
                      font=font, 
                      bg='#AAC7D7', 
                      fg=fg_color, 
                      cursor="hand2", 
                      borderwidth=10, 
                      relief=RAISED, 
                      padx=10, 
                      pady=10,
                      width=width,
                      height=height
                      )
        
        # Places the label in the grid layout at the specified row/col
        label.grid(row=row, column=column, columnspan=colspan, sticky="w", padx=5, pady=5)
        
        # If label_key provided, makes the label clickable to change its color
        if label_key:
            label.bind("<Button-1>", lambda e: choose_color(label_key, label))
            
        # Returns the created label
        return label

    # Title Label
    create_label("Fitness for All!", 0, 0, label_key='title_label', colspan=4, width=100)   
        
    # Name Label
    create_label("Name:", 1, 0, label_key='name_label')
    create_label(exercise.name, 1, 1, font=("Verdana", 15, 'bold'), label_key='name')

    # Description Label
    create_label("Description:", 2, 0, label_key='description_label')
    create_label(exercise.description, 2, 1, label_key='description')
    
    # Duration Label
    create_label("Duration:", 3, 0, label_key='duration_label')
    create_label(str(exercise.duration) + " seconds", 3, 1, label_key='duration')
    
    # Difficulty Label
    create_label("Difficulty:", 4, 0, label_key='difficulty_label')
    create_label(exercise.difficulty, 4, 1, label_key='difficulty')
    
    # Muscle Groups Label
    create_label("Muscles Worked:", 5, 0, label_key='muscles_label')
    muscle_groups_str = ", ".join(exercise.muscle_groups)
    create_label(muscle_groups_str, 5, 1, label_key='muscles')
    
    # Instructions Label
    instructions_label = create_label("Instructions:", 6, 0, label_key='instructions_label')
    instructions_text = Text(window, height=6, width=40, wrap=WORD, 
                             fg=text_colors[exercise.name]['instructions'], 
                             cursor="hand2", 
                             borderwidth=10, 
                             relief=SUNKEN
                             )
    instructions_text.insert(END, exercise.instructions)
    instructions_text.config(state=DISABLED, font=("Verdana", 15, 'bold'))  # Make the text entry field read-only
    instructions_text.grid(row=6, column=1, sticky="w")
    instructions_text.bind("<Button-1>", lambda e: choose_color('instructions', instructions_text))
    
    # Click on text Label
    create_label("Click on any text to change its color :D", 7, 1, font=("Verdana", 13), label_key='textcolor', colspan=1)
    
    

    # Load/Displaying the Workout Images
    img = Image.open(exercise.image_path)
    img = img.resize((300, 300), Image.LANCZOS)
    photo_img = ImageTk.PhotoImage(img)  # Converts PIL image object into PhotoImage 
    img_label = Label(window, image=photo_img)
    img_label.image = photo_img  # Keeps a reference to avoid garbage collection
    img_label.grid(row=1, column=2, rowspan=6)
    
    # Creating Navigation Buttons
    prev_btn = Button(window, text="Previous", 
                      command=show_prev_exercise,
                      width=15,
                      height=5,
                      cursor="left_ptr",
                      relief=RAISED,
                      borderwidth=10,
                      font=("Verdana", 13)
                      )
    prev_btn.grid(row=7, column=0, sticky="w", padx=5, pady=30)
    
    next_btn = Button(window, text="Next", 
                      command=show_next_exercise,
                      width=15,
                      height=5,
                      cursor="right_ptr",
                      relief=RAISED,
                      borderwidth=10,
                      font=("Verdana", 13),
                      activebackground="yellow"
                      )
    next_btn.grid(row=7, column=1, sticky="e")

# Function to show previous exercise
def show_prev_exercise():
    global current_index
    current_index = (current_index - 1) % len(exercises)
    display_exercise_details()

# Function to show next exercise
def show_next_exercise():
    global current_index
    current_index = (current_index + 1) % len(exercises)
    display_exercise_details()    

# Displaying the first exercise
display_exercise_details()

# Configure grid row and column weights to make them expandable
for row in range(8):
    window.grid_rowconfigure(row, weight=1)
for column in range(3):
    window.grid_columnconfigure(column, weight=1)


window.mainloop()

In [3]:
from tkinter import *
from tkinter import colorchooser
from PIL import ImageTk, Image

# Assuming these imports are already done
%run ./base_exercises.ipynb
%run ./exercise_data.ipynb

# Global variable to keep track of the current exercise index
current_index = 0

# Dictionary to store text colors for each exercise
text_colors = {}

# Dictionary to store current colors for different parts of the exercise details
current_colors = {
    'title_label': 'black',
    'name_label': 'black',
    'name': 'black',
    'description_label': 'black',
    'description': 'black',
    'duration_label': 'black',
    'duration': 'black',
    'difficulty_label': 'black',
    'difficulty': 'black',
    'muscles_label': 'black',
    'muscles': 'black',
    'instructions_label': 'black',
    'instructions': 'black',
    'textcolor': 'red'
}

# Create the main window
window = Tk()
window.title("Background Test")
window.geometry("800x700")

# Load the background image
image_path = "../Shared Components/Assets/puzzle_bg.png"
bg_image = PhotoImage(file=image_path)
bg_label = Label(window, image=bg_image)
bg_label.place(relwidth=1, relheight=1)
bg_label.image = bg_image  # Keep a reference to avoid garbage collection


# Function to display exercise details in the window
def display_exercise_details():
    global current_index
    
    exercise = exercises[current_index]
    
    # Initialize color dictionary for current exercise if not already done
    if exercise.name not in text_colors:
        text_colors[exercise.name] = current_colors.copy()  # Creates a new dict with default colors for the current exercise
    
    # Clear previous labels and texts
    for widget in window.winfo_children():
        if not isinstance(widget, Label):  # Skip the background label
            widget.destroy()
    
    # Function to choose color
    def choose_color(label_key, label):
        color_code = colorchooser.askcolor(title="Choose Text Color")[1]
        if color_code:
            label.config(fg=color_code)
            text_colors[exercise.name][label_key] = color_code
            current_colors[label_key] = color_code
    
    # Create labels to display exercise details
    def create_label(text, row, column, font=("Verdana", 15, 'bold'), label_key=None, colspan=1, width=None, height=None):
        # Initialize exercise name in text_colors if not already initialized
        if exercise.name not in text_colors:
            text_colors[exercise.name] = current_colors.copy()
        
        # Determine the text color to use for this label
        fg_color = text_colors[exercise.name].get(label_key, current_colors.get(label_key, 'black'))
        
        # Create the label with the specified text, font, and color
        label = Label(window, text=text, font=font, bg='#AAC7D7', fg=fg_color, cursor="hand2", 
                      borderwidth=10, relief=RAISED, padx=10, pady=10, width=width, height=height)
        
        # Place the label in the grid layout at the specified row/column
        label.grid(row=row, column=column, columnspan=colspan, sticky="w", padx=5, pady=5)
        
        # If label_key provided, make the label clickable to change its color
        if label_key:
            label.bind("<Button-1>", lambda e: choose_color(label_key, label))
        
        return label
    
    # Example labels (adjust according to your layout)
    create_label("Fitness for All!", 0, 0, label_key='title_label', colspan=4, width=100)
    create_label("Name:", 1, 0, label_key='name_label')
    create_label(exercise.name, 1, 1, font=("Verdana", 15, 'bold'), label_key='name')
    create_label("Description:", 2, 0, label_key='description_label')
    create_label(exercise.description, 2, 1, label_key='description')
    create_label("Duration:", 3, 0, label_key='duration_label')
    create_label(str(exercise.duration) + " seconds", 3, 1, label_key='duration')
    create_label("Difficulty:", 4, 0, label_key='difficulty_label')
    create_label(exercise.difficulty, 4, 1, label_key='difficulty')
    create_label("Muscles Worked:", 5, 0, label_key='muscles_label')
    muscle_groups_str = ", ".join(exercise.muscle_groups)
    create_label(muscle_groups_str, 5, 1, label_key='muscles')
    instructions_label = create_label("Instructions:", 6, 0, label_key='instructions_label')
    
    instructions_text = Text(window, height=6, width=40, wrap=WORD, 
                             fg=text_colors[exercise.name].get('instructions', current_colors['instructions']), 
                             cursor="hand2", 
                             borderwidth=10, 
                             relief=SUNKEN)
    instructions_text.insert(END, exercise.instructions)
    instructions_text.config(state=DISABLED, font=("Verdana", 15, 'bold'))
    instructions_text.grid(row=6, column=1, sticky="w")
    instructions_text.bind("<Button-1>", lambda e: choose_color('instructions', instructions_text))
    
    create_label("Click on any text to change its color :D", 7, 1, font=("Verdana", 13), label_key='textcolor', colspan=1)
    
    # Load/Displaying the Workout Images

    img = Image.open(exercise.image_path)
    img = img.resize((300, 300), Image.LANCZOS)
    photo_img = ImageTk.PhotoImage(img)
    img_label = Label(window, image=photo_img)
    img_label.image = photo_img
    img_label.grid(row=1, column=2, rowspan=6)

    
    # Creating Navigation Buttons
    prev_btn = Button(window, text="Previous", command=show_prev_exercise, width=15, height=5,
                      cursor="left_ptr", relief=RAISED, borderwidth=10, font=("Verdana", 13))
    prev_btn.grid(row=7, column=0, sticky="w", padx=5, pady=30)
    
    next_btn = Button(window, text="Next", command=show_next_exercise, width=15, height=5,
                      cursor="right_ptr", relief=RAISED, borderwidth=10, font=("Verdana", 13), activebackground="yellow")
    next_btn.grid(row=7, column=1, sticky="e")

# Function to show previous exercise
def show_prev_exercise():
    global current_index
    current_index = (current_index - 1) % len(exercises)
    display_exercise_details()

# Function to show next exercise
def show_next_exercise():
    global current_index
    current_index = (current_index + 1) % len(exercises)
    display_exercise_details()

# Initial display of exercise details
display_exercise_details()

# Configure grid row and column weights to make them expandable
for row in range(8):
    window.grid_rowconfigure(row, weight=1)
for column in range(3):
    window.grid_columnconfigure(column, weight=1)

# Start the main loop
window.mainloop()

